In [2]:
import cv2
import numpy as np
import csv

video_path = '/Users/zzze/Downloads/Q5_6644_argue_and_door_slam.mp4'
cap = cv2.VideoCapture(video_path)

def detect_shapes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    shapes = []
    for contour in contours:
        if cv2.contourArea(contour) > 40:  # Can change to other numbers, this is just filtering small contours.
            approx = cv2.approxPolyDP(contour, 0.04 * cv2.arcLength(contour, True), True)
            if len(approx) == 3:
                shape_type = "Triangle"
            elif len(approx) == 4:
                shape_type = "Quadrilateral"
            elif len(approx) > 4:
                shape_type = "Circle"
            else:
                shape_type = "Unknown"
            shapes.append((contour, shape_type))
    return shapes

class BoundaryTracker:
    def __init__(self):
        self.nextObjectID = 0
        self.objects = {}
        self.disappeared = {}

    def register(self, contour):
        self.objects[self.nextObjectID] = contour
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, inputContours):
        if len(inputContours) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > 50:
                    self.deregister(objectID)
            return self.objects

        if len(self.objects) == 0:
            for contour in inputContours:
                self.register(contour)
        else:
            objectIDs = list(self.objects.keys())
            objectContours = list(self.objects.values())
            inputCentroids = [np.mean(contour, axis=0)[0] for contour in inputContours]
            objectCentroids = [np.mean(contour, axis=0)[0] for contour in objectContours]

            D = np.linalg.norm(np.array(objectCentroids)[:, np.newaxis] - np.array(inputCentroids), axis=2)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                objectID = objectIDs[row]
                self.objects[objectID] = inputContours[col]
                self.disappeared[objectID] = 0
                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    if self.disappeared[objectID] > 50:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputContours[col])

        return self.objects

tracker = BoundaryTracker()
frame_count = 0
position_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    shapes = detect_shapes(frame)

    for contour, shape_type in shapes:
        cv2.putText(frame, shape_type, (contour[0][0][0], contour[0][0][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
    
    objects = tracker.update([contour for contour, shape_type in shapes])

    for objectID, contour in objects.items():
        contour_points = contour.reshape(-1, 2).tolist()
        position_data.append([frame_count, objectID, contour_points])

    cv2.imshow("Frame", frame)
    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

with open('position2_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Frame", "ObjectID", "ContourPoints"])
    writer.writerows(position_data)

print("Data exported to position_data.csv")


Data exported to position_data.csv


: 